In [1]:
import Annotator
import gff_helpers
import gffutils
import region_helpers as r
import pybedtools
import os
from tqdm import tnrange, tqdm_notebook

In [2]:
db_file = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
# db_file = '/projects/ps-yeolab3/bay001/annotations/gencode.vM10.annotation.db'

In [3]:
transcripts_dict = {'transcript_1':{'start':10, 'end':20}}
exons_dict = {'transcript_1':{'start':12, 'end':15}}

In [4]:
d = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
bed = os.path.join(d,'WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed.brianannotated2')

In [5]:
a = Annotator.Annotator(db_file, chroms=['chr7'])

Initializing/creating defs: 100%|██████████| 8/8 [01:46<00:00, 15.82s/it]

In [6]:
positions = []
for exon in e['ENST00000542080.1']:
    positions.append(exon['start'])
    positions.append(exon['end'])
sorted(positions)

NameError: name 'e' is not defined

In [ ]:
t = a._get_all_transcripts_dict()

In [ ]:
e['ENST00000542080.1']

In [ ]:
def find_introns(transcript, exons):
    positions = []
    introns = []
    for exon in exons:
        positions.append(exon['start']-1)
        positions.append(exon['end']+1)
    positions = sorted(positions)
    
    if positions[0] < transcript['start']: # there is no intron at the start of the feature
        positions.pop(0)
    else:
        positions.insert(transcript['start'])
    if positions[-1] > transcript['end']:
        positions.pop(-1)
    else:
        positions.append(transcript['end'])
    for i in range(0, len(positions)-1, 2):
        introns.append({'start':positions[i], 'end':positions[i+1]})
    return introns
find_introns(t['ENST00000542080.1'][0],e['ENST00000542080.1'])

In [ ]:
for key, features in a.features_dict.iteritems():
    for feature in features:
        if feature.featuretype == 'transcript':
            exons = a.exons_dict[feature.attributes['transcript_id'][0]]
            print(feature.attributes['transcript_id'])
    break

In [ ]:
exons

In [14]:
import pandas as pd

bed = '/home/bay001/projects/codebase/bfx/pyscripts/data/WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed.brianannotated2'
df = pd.read_table(bed, names=['chrom','start','end','pv','fc','strand','annotation'])
def return_overlapping_features(row):
    return row['annotation'].split('|')

df['features'] = df.apply(return_overlapping_features, axis=1)

In [17]:
for i in range(0, 30): # for each line (peak)
    features_test = df.ix[i]['features']
    extended_features = []
    for feature in features_test: # for each list of features
        print(feature)
        transcript, start, end, strand, feature_type, gene_id, gene_name, transcript_type_list = feature.split(':')
        transcript_types = transcript_type_list.split(',')
        for transcript_type in transcript_types:
            extended_features.append(
                '{}:{}:{}:{}:{}:{}:{}'.format(
                    transcript, start, end, strand, feature_type,
                    gene_id, transcript_type
                )
            )
            print('{}:{}:{}:{}:{}:{}:{}'.format(
                    transcript, start, end, strand, feature_type,
                    gene_id, transcript_type
                ))
    print("____")

ENST00000359863.4:98476113:98610866:+:transcript:ENSG00000196367.8:TRRAP:protein_coding
ENST00000359863.4:98476113:98610866:+:transcript:ENSG00000196367.8:protein_coding
ENST00000359863.4:98478874:98479597:+:intron:ENSG00000196367.8:TRRAP:protein_coding
ENST00000359863.4:98478874:98479597:+:intron:ENSG00000196367.8:protein_coding
ENST00000417523.1:98475556:98488022:+:transcript:ENSG00000196367.8:TRRAP:protein_coding
ENST00000417523.1:98475556:98488022:+:transcript:ENSG00000196367.8:protein_coding
ENST00000417523.1:98478874:98479597:+:intron:ENSG00000196367.8:TRRAP:protein_coding
ENST00000417523.1:98478874:98479597:+:intron:ENSG00000196367.8:protein_coding
ENSG00000196367.8:98475556:98610866:+:gene:ENSG00000196367.8:TRRAP:protein_coding
ENSG00000196367.8:98475556:98610866:+:gene:ENSG00000196367.8:protein_coding
ENST00000355540.3:98476113:98610866:+:transcript:ENSG00000196367.8:TRRAP:protein_coding
ENST00000355540.3:98476113:98610866:+:transcript:ENSG00000196367.8:protein_coding
ENST0000

ENST00000359863.4:98576392:98576527:+:CDS:ENSG00000196367.8:TRRAP:protein_coding
ENST00000359863.4:98576392:98576527:+:CDS:ENSG00000196367.8:protein_coding
ENSG00000196367.8:98475556:98610866:+:gene:ENSG00000196367.8:TRRAP:protein_coding
ENSG00000196367.8:98475556:98610866:+:gene:ENSG00000196367.8:protein_coding
ENST00000355540.3:98576392:98576527:+:exon:ENSG00000196367.8:TRRAP:protein_coding
ENST00000355540.3:98576392:98576527:+:exon:ENSG00000196367.8:protein_coding
ENST00000355540.3:98576392:98576527:+:CDS:ENSG00000196367.8:TRRAP:protein_coding
ENST00000355540.3:98576392:98576527:+:CDS:ENSG00000196367.8:protein_coding
ENST00000355540.3:98476113:98610866:+:transcript:ENSG00000196367.8:TRRAP:protein_coding
ENST00000355540.3:98476113:98610866:+:transcript:ENSG00000196367.8:protein_coding
ENST00000446306.3:98576392:98576527:+:exon:ENSG00000196367.8:TRRAP:protein_coding
ENST00000446306.3:98576392:98576527:+:exon:ENSG00000196367.8:protein_coding
ENST00000446306.3:98576392:98576527:+:CDS:EN

In [ ]:
def prioritize(
            self, features, v=True, transcript_priority=PRIORITY):
        """
        Groups each transcript feature into genes.
        """
        hi = []  # contains highest priority genic feature for each gene
        genes = defaultdict(list)
        for feature in features:
            for gene_id in feature.attributes['gene_id']:
                genes[gene_id].append(feature) # append features to their respective genes

        for gene, features in genes.iteritems(): # for each transcript, iterate through feature list and prioritize
            for feature in features:
                if 'protein_coding' not in feature.attributes['transcript_type']:
                    if v == True:
                        print(feature.attributes['transcript_type'])
                    if feature.featuretype == 'exon' or feature.featuretype == 'UTR':
                        feature.featuretype = 'NONCODING_EXON'
                    elif feature.featuretype == 'gene':
                        feature.featuretype = 'NONCODING_INTRON'
            features.sort(key=lambda x: transcript_priority.index(x.featuretype.upper()))
        for gene_id, overlapping_features in genes.iteritems(): # iterate over each gene and prioritize according to genic region.
            if v == True:
                print(gene_id)
                for feature in overlapping_features:
                    print(feature.featuretype, feature.attributes['transcript_id'], feature.attributes['gene_id'], feature.start, feature.end)
            first_priority = overlapping_features[0]
            if first_priority.featuretype == 'UTR':
                annotation = self._classify_utr(first_priority)
            else:
                annotation = first_priority # .featuretype.replace('gene','INTRON')
            hi.append('{}|{}'.format(annotation, gene_id))
        return hi